## 0. Load the Data

In [1]:
! pip install keras_applications

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.3 MB/s 


In [2]:
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
import keras_applications
# from keras.applications.resnet50 import ResNet50, decode_predictions
# https://github.com/keras-team/keras-contrib/issues/283
from keras_applications.resnet50 import ResNet50, decode_predictions
from keras import layers
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping
import tensorflow as tf
import os

In [3]:
from keras import optimizers

In [4]:
import zipfile
with zipfile.ZipFile("viziometrics.zip",'r') as zip_ref:
  zip_ref.extractall("./")

In [61]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/train/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/val/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/test/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 2733 images belonging to 5 classes.
Found 1571 images belonging to 5 classes.
Found 1563 images belonging to 5 classes.


## 1. Feature Extraction 

In [62]:
# download the pre-trained ResNet50 model
resnet = keras.applications.resnet.ResNet50(weights='imagenet', input_shape=(224,224,3))

# inspect the ResNet50 architecture
resnet.summary()


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [63]:
# The "embedding layer" is the "avg_pool" layer, the layer before the predictions. 
# We want the outputs from this layer

layer_name = 'avg_pool'
resnet_embedder = Model(inputs=resnet.input,outputs=resnet.get_layer(layer_name).output)

# observe that this layer has output shape 2048
# let's save this dimension for later use
DIM = 2048

- Use ResNet50 to extract features from the images

In [64]:
# # use ResNet50 to extract features from the images
#train_features = resnet.predict(train_generator, verbose=1) # takes ~4min on my laptop
#val_features = resnet.predict(val_generator, verbose=1) # 2min
#test_features = resnet.predict(test_generator, verbose=1) # 2min

train_features = resnet_embedder.predict(train_generator, verbose=1)
val_features = resnet_embedder.predict(val_generator, verbose=1)
test_features = resnet_embedder.predict(test_generator, verbose=1)


print(train_features.shape, val_features.shape, test_features.shape)

1563/1563 [==============================] - 13s 8ms/step
(2733, 2048) (1571, 2048) (1563, 2048)


In [65]:
# reshape the features to 2D arrays
train_X = train_features.reshape((-1, DIM))
val_X = val_features.reshape((-1, DIM))
test_X = test_features.reshape((-1, DIM))

print(train_X.shape, val_X.shape, test_X.shape)

(2733, 2048) (1571, 2048) (1563, 2048)


In [66]:
# # read the class labels from the generators
train_labels = train_generator.classes
val_labels = val_generator.classes
test_labels = test_generator.classes

print(test_labels)

[0 0 0 ... 4 4 4]


In [67]:
# # get one-hot encoding of labels
def get_one_hot(labels, nb_classes):
     res = np.eye(nb_classes)[np.array(labels).reshape(-1)]
     return res.reshape(list(labels.shape)+[nb_classes])

# # use get_one_hot()
NUM_CLASSES = 5
train_y = get_one_hot(train_labels, NUM_CLASSES)
val_y = get_one_hot(val_labels, NUM_CLASSES)
test_y = get_one_hot(test_labels, NUM_CLASSES)

In [68]:
train_y.shape

(2733, 5)

In [69]:
val_y.shape

(1571, 5)

In [70]:
test_y.shape

(1563, 5)

In [71]:
# create our model: a not-so-deep neural network
model = Sequential()

# input layer takes arrays of shape (*, DIM)
# todo: please modify this network to define your own model
model.add(layers.Dense(300, activation = "relu", input_shape=(DIM,))) 
model.add(layers.Dropout(0.1, noise_shape=None, seed=None))
model.add(layers.Dense(NUM_CLASSES, activation = "softmax"))

**I modified the parameters of dense layers and dropout layers**

In [72]:
# print out network architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 300)               614700    
                                                                 
 dropout_4 (Dropout)         (None, 300)               0         
                                                                 
 dense_5 (Dense)             (None, 5)                 1505      
                                                                 
Total params: 616,205
Trainable params: 616,205
Non-trainable params: 0
_________________________________________________________________


In [73]:
# # you need to compile the model before you can train it
model.compile(optimizer= 'sgd',loss='categorical_crossentropy',metrics=['acc'])

In [74]:
# # save model weights while model is under training
checkpoint_filepath = 'model.{epoch:02d}-{val_loss:.2f}.h5'

# # train the model for about 20 epochs
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

EPOCHS = 20

model.fit(x = train_X, y = train_y, epochs = EPOCHS, validation_data = (val_X, val_y),callbacks = [model_checkpoint_callback])

model.save('model.{epoch:02d}-{val_loss:.2f}.h5')

Epoch 1/20
86/86 [==============================] - 1s 5ms/step - loss: 0.4761 - acc: 0.8536 - val_loss: 1.0576 - val_acc: 0.6321
Epoch 2/20
86/86 [==============================] - 0s 3ms/step - loss: 0.2694 - acc: 0.9162 - val_loss: 0.8885 - val_acc: 0.6289
Epoch 3/20
86/86 [==============================] - 0s 3ms/step - loss: 0.2263 - acc: 0.9232 - val_loss: 0.7901 - val_acc: 0.6913
Epoch 4/20
86/86 [==============================] - 0s 3ms/step - loss: 0.1976 - acc: 0.9374 - val_loss: 0.6993 - val_acc: 0.7218
Epoch 5/20
86/86 [==============================] - 0s 3ms/step - loss: 0.1824 - acc: 0.9393 - val_loss: 0.8499 - val_acc: 0.6645
Epoch 6/20
86/86 [==============================] - 0s 3ms/step - loss: 0.1695 - acc: 0.9437 - val_loss: 0.8232 - val_acc: 0.6747
Epoch 7/20
86/86 [==============================] - 0s 3ms/step - loss: 0.1601 - acc: 0.9469 - val_loss: 0.8152 - val_acc: 0.6913
Epoch 8/20
86/86 [==============================] - 0s 3ms/step - loss: 0.1524 - acc: 0.94

In [ ]:
# The best train accuracy is 99.07%
# The best validation accuracy is 75.81%.

In [75]:
# # load the weights of your best model
model.load_weights(checkpoint_filepath)

# # measure test accuracy
scores = model.evaluate(test_X,test_y, verbose = 1)
print('Test accuracy: ',model.metrics_names[1], round(scores[1]*100,2),'%')

49/49 [==============================] - 0s 2ms/step - loss: 0.7365 - acc: 0.7441
Test accuracy:  acc 74.41 %


## 2. Fine-tuning 

In [42]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/train/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/val/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=True,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/test/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 2733 images belonging to 5 classes.
Found 1571 images belonging to 5 classes.
Found 1563 images belonging to 5 classes.


In [43]:
# # add custom prediction layers to ResNet50
x = resnet.output 
# x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.1)(x)
predictions = layers.Dense(5, activation= 'softmax')(x)
model = Model(inputs = resnet.input, outputs = predictions)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [44]:
# # need to compile the model before training
model.compile(optimizer= 'sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [45]:
# # train the model for about 10 epochs
# # each epoch takes about 800 seconds on my laptop
# STEP_SIZE_TRAIN=train_generator.n/train_generator.batch_size
# STEP_SIZE_VAL=val_generator.n/val_generator.batch_size
# model.fit_generator(generator=...,
#                     steps_per_epoch=...,
#                     validation_data=...,
#                     validation_steps=...,
#                     epochs=...
# )

In [46]:
# # save model weights while model is under training
checkpoint_filepath = 'model.{epoch:02d}-{val_loss:.2f}.h5'

# # train the model for about 10 epochs
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


In [47]:
STEP_SIZE_TRAIN=train_generator.n/train_generator.batch_size
STEP_SIZE_VAL=val_generator.n/val_generator.batch_size

In [48]:
model.fit_generator(generator = train_generator,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_data = val_generator,
                    validation_steps = STEP_SIZE_VAL,
                    epochs = 10,
                    callbacks = [model_checkpoint_callback])
          
model.save('model.{epoch:02d}-{val_loss:.2f}.h5')

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


85/85 [==============================] - 23s 227ms/step - loss: 1.4793 - accuracy: 0.4687 - val_loss: 1.5372 - val_accuracy: 0.2342
Epoch 2/10
85/85 [==============================] - 18s 210ms/step - loss: 1.2613 - accuracy: 0.5763 - val_loss: 1.4886 - val_accuracy: 0.2342
Epoch 3/10
85/85 [==============================] - 18s 213ms/step - loss: 1.1158 - accuracy: 0.8280 - val_loss: 1.4610 - val_accuracy: 0.5041
Epoch 4/10
85/85 [==============================] - 18s 212ms/step - loss: 0.9998 - accuracy: 0.8328 - val_loss: 1.4378 - val_accuracy: 0.5054
Epoch 5/10
85/85 [==============================] - 18s 208ms/step - loss: 0.9166 - accuracy: 0.8408 - val_loss: 1.4175 - val_accuracy: 0.5048
Epoch 6/10
85/85 [==============================] - 18s 208ms/step - loss: 0.8440 - accuracy: 0.8423 - val_loss: 1.4007 - val_accuracy: 0.5035
Epoch 7/10
85/85 [==============================] - 18s 208ms/step - loss: 0.7887 - accuracy: 0.8335 - val_loss: 1.4841 - val_accuracy: 0.3870
Epoch 8/10

In [60]:
# # load the weights of your best model
model.load_weights('model.08-1.39.h5')

# # measure test accuracy
pred_softmax = model.predict_generator(test_generator, steps = test_generator.n)
pred_class = np.argmax(pred_softmax, axis=-1)
test_acc = np.sum(pred_class == test_generator.classes) / test_generator.n
print('Test accuracy: ', round(test_acc*100,2),'%')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  """


Test accuracy:  48.82 %
